In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
movies = pd.read_csv("data.txt")

In [5]:
movies["Clean_Rating"] = movies["Rating"].str.replace("Five", "5", regex=True)
movies["Clean_Rating"] = movies["Clean_Rating"].str.replace(r'[^0-9,.]', '', regex=True)


In [7]:
movies = movies[movies["User"] != "Victor"]

In [9]:
movies["Clean_Rating"] = movies["Clean_Rating"].fillna(0)

/tmp/ipykernel_434997/811439065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["Clean_Rating"] = movies["Clean_Rating"].fillna(0)


In [11]:
movies["Clean_Rating"] = movies["Clean_Rating"].astype(float)

/tmp/ipykernel_434997/457608370.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["Clean_Rating"] = movies["Clean_Rating"].astype(float)


In [12]:
movies = movies.reset_index(drop=True)

In [13]:
movies

,User,Movie,Rating,Clean_Rating
0,Alice,Star Wars,5,5.0
1,Frank,The Godfather,4,4.0
2,Bob,Titanic,5,5.0
3,Carol,The Matrix,3,3.0
4,Dave,Inception,2,2.0
5,Emily,Pulp Fiction,4,4.0
6,Quincy,Star Wars,5,5.0
7,Bob,Star Wars,2,2.0
8,Frank,Forrest Gump,3,3.0
9,Alice,The Matrix,4,4.0


In [15]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf = vectorizer.fit_transform(movies["Movie"])

In [16]:
def search_movie(title):
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    movie_results = movies.iloc[indices][::-1]
    return movie_results

In [17]:
search_movie("star wars")

,User,Movie,Rating,Clean_Rating
55,Zane,Star Wars,4,4.0
23,Frank,Star Wars,5,5.0
30,Nina,Star Wars,5,5.0
32,Sarah,Star Wars,5x,5.0
6,Quincy,Star Wars,5,5.0


In [25]:
# vectorizer2 = TfidfVectorizer(ngram_range=(1,2))
tfidf2 = vectorizer.fit_transform(movies["User"])

In [26]:
def search_user(user):
    query_vec2 = vectorizer.transform([user])
    similarity = cosine_similarity(query_vec2, tfidf2).flatten()
    indices = np.argpartition(similarity, -5)[-10:]
    user_results = movies.iloc[indices][::-1]
    return user_results

In [27]:
search_user("zane")

,User,Movie,Rating,Clean_Rating
55,Zane,Star Wars,4,4.0
47,Zane,Forrest Gump,3.5,3.5
42,Zane,Pulp Fiction,3.5,3.5
22,Bob,The Godfather,4,4.0
17,Ivy,Star Wars,5,5.0
18,Emily,Forrest Gump,3,3.0
16,Carol,Pulp Fiction,4,4.0
15,Paul,The Godfather,5,5.0
23,Frank,Star Wars,5,5.0
19,Quincy,The Matrix,4,4.0


In [55]:
 def user_recommended_movies(user):
    # user_2 = "Frank
    user_watching = movies[(movies["User"] == user) & (movies["Clean_Rating"] >= 3)]
    ## Similar users with ratings over 3 for same movies user likes
    users_watching_similar_movies = movies[(movies["Movie"].isin(user_watching["Movie"])) & (movies["Clean_Rating"] >= 3)]
    # ## We have to find other movies these users have liked that have ratings of >= 3
    similar_user_recs = movies[(movies["User"].isin(users_watching_similar_movies["User"])) & (movies["Clean_Rating"] >= 3)][["Movie", "User"]]
    user_recs = similar_user_recs[(similar_user_recs["User"] != user)]["Movie"].unique()
    # similar_user_recs
    user_recs_df = pd.DataFrame(user_recs.tolist(), columns=["'s Recommended Movies"])
    user_recs_df = user_recs_df.add_prefix(user)
    
    return user_recs_df

In [57]:
user_recommended_movies("Zane")

,Zane's Recommended Movies
0,Star Wars
1,The Godfather
2,The Matrix
3,Pulp Fiction
4,Forrest Gump
5,Inception
6,Titanic


In [102]:
def find_similar_movies(movie):
    #Finding recommendations from similar users
    similar_users = movies[(movies["Movie"] == movie) & (movies["Clean_Rating"] >= 3)]["User"].unique()
    similar_user_recs = movies[(movies["User"].isin(similar_users)) & (movies["Clean_Rating"] >= 3)]["Movie"]
    
    #Only 10% of users
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
   
    movie_recs_df = pd.DataFrame(similar_user_recs)
    movie_recs_df = movie_recs_df.drop(columns="count")
    movie_recs_df.reset_index(inplace=True)
    movie_recs_df = movie_recs_df.rename(columns = {'Movie':'Movies similar to '})
    movie_recs_df = movie_recs_df.add_suffix(movie)
    
    return movie_recs_df[1:]

In [104]:
find_similar_movies("The Godfather")

,Movies similar to The Godfather
1,Forrest Gump
2,Titanic
3,Pulp Fiction
4,Star Wars


In [90]:
ans

,countTitanic
Movie,
Titanic,1.0
The Godfather,0.4
Forrest Gump,0.4


In [82]:
ans.reset_index(inplace=True)
ans = ans.rename(columns = {'Movie':'Movies similar to '}, inplace = True)